In [18]:
# 파일 업로드

from google.colab import files
uploaded = files.upload()


Saving customer (1).xlsx to customer (1).xlsx


In [19]:
# 파일 업로드

from google.colab import files
uploaded = files.upload()

Saving sales (1).csv to sales (1).csv


In [20]:
# 파일 불러오기

import pandas as pd
import numpy as np
sales = pd.read_csv( './sales.csv', encoding='utf-8')
customer = pd.read_excel('./customer.xlsx')
print(sales.head())
print(customer.head())

      purchase_date item_name  item_price customer_name
0  2019-06-13 18:02       상품A       100.0           김가온
1  2019-07-13 13:05     상 품 S         NaN           김우찬
2  2019-05-11 19:42     상 품 a         NaN           김유찬
3  2019-02-12 23:40       상품Z      2600.0           김재현
4   2019-04-22 3:09       상품a         NaN           김강현
   고객이름  지역                  등록일
0  김 현성  H시  2018-01-04 00:00:00
1  김 도윤  E시                42782
2  김 지한  A시  2018-01-07 00:00:00
3  김 하윤  F시                42872
4  김 시온  E시                43127


In [21]:
# 상품명 오류 수정 : A~Z 까지 26개만 있어야 함, 빈칸 제거, 대문자 변환

sales['item_name'] = sales['item_name'].str.upper()
sales['item_name'] = sales['item_name'].str.replace(' ','')
pd.unique(sales['item_name'])

array(['상품A', '상품S', '상품Z', '상품V', '상품O', '상품U', '상품L', '상품C', '상품I',
       '상품R', '상품X', '상품G', '상품P', '상품Q', '상품Y', '상품N', '상품W', '상품E',
       '상품K', '상품B', '상품F', '상품D', '상품M', '상품H', '상품T', '상품J'],
      dtype=object)

In [22]:
# 구매년월 새로 컬럼
sales['month'] = pd.to_datetime(sales['purchase_date']).dt.strftime('%Y%M')
print(sales['month'])

0       201902
1       201905
2       201942
3       201940
4       201909
         ...  
2994    201956
2995    201903
2996    201914
2997    201956
2998    201931
Name: month, Length: 2999, dtype: object


In [23]:
# 등록일 변경

date = customer['등록일'].astype('str').str.isdigit()
date.sum()
newDate1 = pd.to_timedelta(customer.loc[date,'등록일'].astype('float'),unit='D') + pd.to_datetime('1900/01/01') # -2
newDate1
newDate2 = pd.to_datetime( customer.loc[~date,'등록일'])
customer['등록일'] = pd.concat([newDate1, newDate2])
customer



,고객이름,지역,등록일
0,김 현성,H시,2018-01-04
1,김 도윤,E시,2017-02-18
2,김 지한,A시,2018-01-07
3,김 하윤,F시,2017-05-19
4,김 시온,E시,2018-01-29
...,...,...,...
195,김 재희,G시,2017-06-20
196,김 도영,E시,2018-06-20
197,김 이안,F시,2017-04-29
198,김 시현,H시,2019-04-19


In [24]:
# purchase_date 수정

date = sales['purchase_date'].astype('str').str.isdigit()
newDate1 = pd.to_timedelta(sales.loc[date,'purchase_date'].astype('float'),unit='D') + pd.to_datetime('1900/01/01') # -2
newDate2 = pd.to_datetime( sales.loc[~date,'purchase_date'])
customer['purchase_date'] = pd.concat([newDate1, newDate2])
sales['month'] = pd.to_datetime(sales['purchase_date']).dt.strftime('%Y%m')
print(sales['month'])


0       201906
1       201907
2       201905
3       201902
4       201904
         ...  
2994    201902
2995    201906
2996    201903
2997    201907
2998    201907
Name: month, Length: 2999, dtype: object


In [26]:
#가격 결측치 수정

sales_null = sales['item_price'].isnull()

for i in list( sales.loc[ sales_null, 'item_name'].unique() ):
  price = sales.loc[ ( ~sales_null ) & (sales['item_name'] == i ), 'item_price'].max()
  sales['item_price'].loc[ (sales_null) & (sales['item_name'] == i )] = price
sales.isnull().sum()

<ipython-input-26-c7a94583b352>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sales['item_price'].loc[ (sales_null) & (sales['item_name'] == i )] = price


purchase_date    0
item_name        0
item_price       0
customer_name    0
month            0
dtype: int64

In [28]:
#고객 이름 수정

customer['고객이름'] = customer['고객이름'].str.replace(' ', '')
customer['고객이름'] = customer['고객이름'].str.replace('  ', '')

In [29]:
#등록월 추출해서 등록 고객수 집계

customer['등록연월'] = customer['등록일'].dt.strftime('%Y%m')
res= customer.groupby('등록연월')['고객이름'].count()
res


등록연월
201701    15
201702    11
201703    14
201704    15
201705    13
201706    14
201707    17
201801    13
201802    15
201803    17
201804     5
201805    19
201806    13
201807    17
201904     2
Name: 고객이름, dtype: int64

In [30]:
#고객 이름을 key로 두 개의 데이터를 결합(join)

joinData = pd.merge(sales, customer, left_on='customer_name', right_on='고객이름', how='left')
joinData = joinData.drop('고객이름', axis=1)
joinData


,purchase_date_x,item_name,item_price,customer_name,month,지역,등록일,purchase_date_y,등록연월
0,2019-06-13 18:02,상품A,100.0,김가온,201906,C시,2017-01-26,2019-05-22 16:50:00,201701
1,2019-07-13 13:05,상품S,1900.0,김우찬,201907,C시,2018-04-07,2019-07-27 00:08:00,201804
2,2019-05-11 19:42,상품A,100.0,김유찬,201905,A시,2018-06-19,2019-06-29 17:42:00,201806
3,2019-02-12 23:40,상품Z,2600.0,김재현,201902,D시,2018-07-22,2019-07-01 11:27:00,201807
4,2019-04-22 3:09,상품A,100.0,김강현,201904,D시,2017-06-07,2019-03-01 16:57:00,201706
...,...,...,...,...,...,...,...,...,...
2994,2019-02-15 2:56,상품Y,2500.0,김정민,201902,B시,2017-07-01,2019-02-20 06:30:00,201707
2995,2019-06-22 4:03,상품M,1300.0,김재원,201906,E시,2018-03-31,2019-06-21 01:54:00,201803
2996,2019-03-29 11:14,상품Q,1700.0,김지율,201903,B시,2017-03-15,2019-07-24 22:05:00,201703
2997,2019-07-14 12:56,상품H,800.0,김승주,201907,E시,2018-07-15,2019-04-18 06:11:00,201807


In [31]:
#덤프파일 저장

joinData.to_csv('dumpData.csv', index=False)

In [ ]:
#데이터분석

joinData.pivot_table(index='month', columns='item_name', values='item_price', aggfunc='sum', fill_value=0)
#고객이름, 지역
joinData.pivot_table(index='month', columns='customer_name', values='item_price', aggfunc='sum', fill_value=0)

In [ ]:
# 구매 이력 없는 고객 정보

away = pd.merge(sales, customer, left_on='customer_name', right_on='고객이름',how='right')
away[away['purchase_date_x'].isnull()][['customer_name','등록일']]